**Exercise 1.**$ max\ 0.85x_1 + 3.75x_2 − 8.75x_3 − 3.45x_4 − 22.38x_5 + 20x_6\\
\text{s.t.} \ − 0.7x_1 − 3.9x_2 − x_3 + x_5 ≤ 14.9\\
\ x_1 − x_4 − x_6 ≤ 0.9\\
−3x_2 − 2x_3 − x_6 ≤ −51.7\\
\ 4.8x_4 + 9.12x_5 + 7.2x_6 ≤ 75\\
−0.8x_1 − 1.5x_2 + 8x_3 − 2x_4 + x_5 ≤ −27,\\
\ x_i ≥ 0 ∀i ∈ {1, 2, . . . , 5}.$

In [60]:
!pip install -q pyomo

In [61]:
from pyomo.environ import *

In [62]:
import numpy as np

In [63]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

In [64]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])


(6, 7)
Num rows: 6
Num cols: 7


In [65]:
print(coef)

[[  0.85   3.75  -8.75  -3.45 -22.38  20.     0.  ]
 [ -0.7   -3.9   -1.     0.     1.     0.    14.9 ]
 [  1.     0.     0.    -1.     0.    -1.     0.9 ]
 [  0.    -3.    -2.     0.     0.    -1.   -51.7 ]
 [  0.     0.     0.     4.8    9.12   7.2   75.  ]
 [ -0.8   -1.5    8.    -2.     1.     0.   -27.  ]]


In [66]:
model1 = ConcreteModel()

In [67]:
N = coef.shape[1]-1
M = coef.shape[0]-1

In [68]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(6,)
[  0.85   3.75  -8.75  -3.45 -22.38  20.  ]


In [69]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[-0.7  -3.9  -1.    0.    1.    0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -3.   -2.    0.    0.   -1.  ]
 [ 0.    0.    0.    4.8   9.12  7.2 ]
 [-0.8  -1.5   8.   -2.    1.    0.  ]]


In [70]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 14.9   0.9 -51.7  75.  -27. ]


In [71]:
lower_bound = np.array([0,0,0,0,0,0])
upper_bound = np.inf

In [72]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [73]:
model1.x = Var(col_indices)

In [74]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [75]:
model1.constraints = ConstraintList()

In [76]:
for i in row_indices:
  model1.constraints.add(summation(constr_coef[i], model1.x) <= constr_rhs[i])

In [77]:
for j in col_indices:
  model1.x[j].setlb(lower_bound[j])
  model1.x[j].setub(upper_bound)

In [78]:
model1.objective = Objective(expr = summation(obj_coef,model1.x), sense = maximize)

In [79]:
model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [80]:
!apt-get install -y -qq coinor-cbc

In [81]:
opt_cbc = SolverFactory('cbc')

In [82]:
result = opt_cbc.solve(model1)
print('Solver status =', result.solver.status)
print('Solver termination condition =', result.solver.termination_condition)
print('Solver termination message =', result.solver.termination_message)

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>
Solver status = warning
Solver termination condition = unbounded
Solver termination message = Model was proven to be unbounded.


The solution comes out to be unbounded .The possible reason for this is that we have not given any upper bounds to decision variables. 

**8)**

In [83]:
model1.objective.set_sense(minimize)

In [84]:
model1.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [85]:
result = opt_cbc.solve(model1)
print('Solver status =', result.solver.status)
print('Solver termination condition =', result.solver.termination_condition)
print('Solver termination message =', result.solver.termination_message)

Solver status = ok
Solver termination condition = optimal
Solver termination message = Model was solved to optimality (subject to tolerances), and an optimal solution is available.


In [86]:
# display solution
print('\nObjective = ', model1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model1.x[i].value)

print('\nConstraints')
model1.constraints.display()


Objective =  -97.02184239599997

Decision Variables
x[ 0 ] =  0.9
x[ 1 ] =  23.002456
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  8.2236842
x[ 5 ] =  0.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


9) When we convert it to minimization problem we have got optimal solution because we defined lower bounds for decision variables.